### Chain-of-Verification Prompting

This prompting technique helps LLMs not to make mistakes by checking their work with questions and answers before giving a final answer. This approach shows that the LLMs can be more accurate and make fewer errors when answering different types of questions or creating long pieces of text. You can find more details here - https://visualsummary.substack.com/p/chain-of-verification-prompting

This notebook is inspired from the below two resources - 
- https://github.com/ritun16/chain-of-verification
- https://github.com/hwchase17/chain-of-verification/tree/master

### Loading libraries 

In [17]:
# Importing the os module for interacting with the operating system
import os

# Import prompts
from prompts import *

# Importing ChatOpenAI model from langchain_openai for AI chat functionalities
from langchain_openai import ChatOpenAI

# Importing PromptTemplate from langchain.prompts for creating prompt templates
from langchain.prompts import PromptTemplate

# Importing ChatPromptTemplate from langchain.prompts for creating chat-specific prompt templates
from langchain.prompts import ChatPromptTemplate

# Importing StrOutputParser from langchain.schema.output_parser for parsing string outputs
from langchain.schema.output_parser import StrOutputParser

# Importing RunnablePassthrough for defining runnable objects that pass through inputs and outputs unchanged
from langchain.schema.runnable import RunnablePassthrough

# Importing RunnableLambda from langchain.schema.runnable for defining runnable objects that execute lambda functions
from langchain.schema.runnable import RunnableLambda

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Setting up LLM

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [3]:
os.environ["OPENAI_API_KEY"] = open('key.txt','r').read()

In [6]:
# Setting up LLM to user
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

### Implementing Chain-of-Verification

![title](chain-of-verification.png)

In [7]:
# Chain to generate initial answer
baseline_response_prompt_template = PromptTemplate.from_template(BASELINE_PROMPT_WIKI)
baseline_response_chain = baseline_response_prompt_template | llm | StrOutputParser()

# Chain to generate a question template for verification answers
verification_question_template_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_TEMPLATE_PROMPT_WIKI)
verification_question_template_chain = verification_question_template_prompt_template | llm | StrOutputParser()

# Chain to generate the verification questionts
verification_question_generation_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_PROMPT_WIKI)
verification_question_generation_chain = verification_question_generation_prompt_template | llm | StrOutputParser()

# Chain to execute the verification
execution_prompt_self_llm = PromptTemplate.from_template(EXECUTE_PLAN_PROMPT_SELF_LLM)
execution_prompt_llm_chain = execution_prompt_self_llm | llm | StrOutputParser()

In [8]:
# Defining a verification chain that splits verification questions into individual lines, 
# processes them through an execution chain, 
# and formats the final response

verification_chain = RunnablePassthrough.assign(
    split_questions=lambda x: x['verification_questions'].split("\n"),
) | RunnablePassthrough.assign(
    answers = (lambda x: [{"verification_question": q} for q in x['split_questions']])| execution_prompt_llm_chain.map()
) | (lambda x: "\n".join(["Question: {} Answer: {}\n".format(question, answer) for question, answer in zip(x['split_questions'],
                                                                                                           x['answers'])]))

# Setting up a chain to generate the final answer using a refined prompt template, processed through LLM and output parsing
final_answer_prompt_template = PromptTemplate.from_template(FINAL_REFINED_PROMPT)
final_answer_chain = final_answer_prompt_template | llm | StrOutputParser()

In [9]:
# Assembling a chain that integrates various components:
# 1. Baseline response generation using a predefined chain
# 2. Verification question template creation via a dedicated chain
# 3. Generation of verification questions through another specialized chain
# 4. Processing verification answers within a verification chain
# 5. Finalizing the answer generation process with a final answer chain

chain = RunnablePassthrough.assign(
    baseline_response=baseline_response_chain
) | RunnablePassthrough.assign(
    verification_question_template=verification_question_template_chain
) | RunnablePassthrough.assign(
    verification_questions=verification_question_generation_chain
) | RunnablePassthrough.assign(
    verification_answers=verification_chain
) | RunnablePassthrough.assign(
    final_answer=final_answer_chain
)

### Response generation

In [10]:
response = chain.invoke({"original_question": "Is Narendra Modi born in Delhi?"})

In [11]:
print(response['original_question'])

Is Narendra Modi born in Delhi?


In [12]:
print(response['baseline_response'])

1. Narendra Modi
2. Delhi


In [13]:
print(response['verification_question_template'])

Was Narendra Modi born in Delhi?


In [14]:
print(response['verification_questions'])

1. Was Narendra Modi born in Delhi?


In [15]:
print(response['verification_answers'])

Question: 1. Was Narendra Modi born in Delhi? Answer: No, Narendra Modi was not born in Delhi. He was born on September 17, 1950, in Vadnagar, a town in the state of Gujarat, India.



In [16]:
print(response['final_answer'])

Final Refined Answer: No, Narendra Modi was not born in Delhi. He was born on September 17, 1950, in Vadnagar, a town in the state of Gujarat, India.
